In [1]:
from fun_utils import *

# Funzione per creare finestre basate sul timestamp
def create_window_timestamp(data, WIN_SIZE, STEP_SIZE):
    windows = []
    start_time = data[0]['Timestamp']  # Timestamp iniziale
    end_time = data[-1]['Timestamp']   # Timestamp finale

    while start_time + WIN_SIZE <= end_time:
        window = [record for record in data if start_time <= record['Timestamp'] < start_time + WIN_SIZE]
        if window:
            windows.append(window)
        start_time += STEP_SIZE

    # Verifica il numero di finestre
    print(f"Numero di finestre create: {len(windows)}")
    
    return windows

# Funzione per calcolate le caratteristiche da una finestra
def get_features(window):
    features = {}
    df_window = pd.DataFrame(window)
    
    # Aggiungi tutte le colonne che devono essere analizzate
    for col in ['GyroX', 'GyroY', 'GyroZ', 'AccX', 'AccY', 'AccZ', 'AccMag', 'AngX', 'AngY', 'AngZ', 'AngMag']:
        if col in df_window.columns:  # Controlla che la colonna esista
            features[f'{col}_mean'] = df_window[col].mean()
            features[f'{col}_std'] = df_window[col].std()
            features[f'{col}_max'] = df_window[col].max()
            features[f'{col}_min'] = df_window[col].min()
            features[f'{col}_range'] = df_window[col].max() - df_window[col].min()
            
            if len(df_window) > 1:
                features[f'{col}_derivative'] = (df_window[col].iloc[-1] - df_window[col].iloc[0]) / (len(df_window) - 1)
        else:
            print(f"Colonna {col} non trovata.")
    
    return features

# Funzione per etichettare le finestre
def set_label(windows):
    labels = []
    for window in windows:
        df_window = pd.DataFrame(window)
        label_counts = df_window['Tag'].value_counts()  # Conta le etichette
        majority_label = -1 if label_counts.get(-1, 0) >= len(df_window) * MAJOR_THRESH else 1
        labels.append(majority_label)
    return labels


df = pd.read_csv('output/Livio_output.csv')

# Calcola la magnitudine dell'accelerazione e degli angoli
df['AccMag'] = np.sqrt(df['AccX']**2 + df['AccY']**2 + df['AccZ']**2)
df['AngMag'] = np.sqrt(df['AngX']**2 + df['AngY']**2 + df['AngZ']**2)

# Parametri di segmentazione
WIN_SIZE = 2000    # Numero di secondi in millisecondi
OVERLAP = 0.5      # Sovrapposizione del 50%
MAJOR_THRESH = 0.5 # soglia di maggioranza
SEC = 1            # Numero di secondi
STEP_SIZE = int(WIN_SIZE * (SEC - OVERLAP))  # Range del passo (500ms in questo caso)


#Dataset in formato lista di dizionari
data = df.to_dict(orient='records')

# Crea finestre basate sul timestamp
windows = create_window_timestamp(data, WIN_SIZE, STEP_SIZE)

# Estrazione delle caratteristiche per ogni finestra
features = [get_features(window) for window in windows]

# Etichettatura delle finestre
labels = set_label(windows)

features_df = pd.DataFrame(features)
features_df['Tag'] = labels

features_df.to_csv('output/Livio_output2.csv', index=False)

Loading BokehJS ...

Numero di finestre create: 660


In [2]:
from sklearn.ensemble import RandomForestClassifier

features_df = pd.read_csv('output/Livio_output2.csv')

X = features_df.drop(columns=['Tag']) 
y = features_df['Tag']  

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

param_grid_rf = {
    'n_estimators': [20, 50, 100, 200],  # Numero di alberi
    'max_depth': [None, 10, 20],      # Profondità massima degli alberi
    'min_samples_split': [2, 5, 10]   # Numero minimo di campioni per dividere un nodo
}

grid_search = GridSearchCV(
    RandomForestClassifier(random_state=42), 
    param_grid_rf, 
    cv=5, 
    scoring='f1', 
    verbose=1
)

grid_search.fit(X_train, y_train)

print("\nMigliori Parametri Trovati:")
print(grid_search.best_params_)

best_model = grid_search.best_estimator_

f1 = evaluate(best_model, X_test, y_test)
print(f"F1 score of the best model on test data: {f1:.2f}%")

Fitting 5 folds for each of 36 candidates, totalling 180 fits

Migliori Parametri Trovati:
{'max_depth': None, 'min_samples_split': 2, 'n_estimators': 50}
Model Performance
Accuracy = 99.49%.
Fbeta = 97.83%.
Recall = 90.00%.
F1 = 94.74%.
Precision = 100.00%.
-----------------------------------
True Positives (TP): 9
False Positives (FP): 0
True Negatives (TN): 188
False Negatives (FN): 1
-----------------------------------
F1 score of the best model on test data: 94.74%


In [3]:
from sklearn.neighbors import KNeighborsClassifier

param_grid_knn = {
    'n_neighbors': [3, 5, 7, 10],  # Numero di vicini
    'weights': ['uniform', 'distance'],  # Tipo di pesatura
    'metric': ['euclidean', 'manhattan']  # Distanza tra i punti
}

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(X_train)
x_test_scaled = scaler.transform(X_test)

grid_search_knn = GridSearchCV(
    KNeighborsClassifier(), 
    param_grid_knn, 
    cv=5, 
    scoring='f1', 
    verbose=1
)

grid_search_knn.fit(x_train_scaled, y_train)

print("\nMigliori Parametri per KNN:")
print(grid_search_knn.best_params_)

best_model = grid_search_knn.best_estimator_

f1 = evaluate(best_model, x_test_scaled, y_test)
print(f"F1 score of the best model on test data: {f1:.2f}%")

Fitting 5 folds for each of 16 candidates, totalling 80 fits

Migliori Parametri per KNN:
{'metric': 'manhattan', 'n_neighbors': 3, 'weights': 'uniform'}
Model Performance
Accuracy = 97.98%.
Fbeta = 88.24%.
Recall = 60.00%.
F1 = 75.00%.
Precision = 100.00%.
-----------------------------------
True Positives (TP): 6
False Positives (FP): 0
True Negatives (TN): 188
False Negatives (FN): 4
-----------------------------------
F1 score of the best model on test data: 75.00%


In [4]:
# Filtra solo i dati normali per il training
X_train_normal = X_train[y_train == 1]
y_train_normal = y_train [y_train == 1]

# Dati esclusi dal training (anomalie)
X_excluded = X_train[y_train == -1]
y_excluded = y_train[y_train == -1]

# Aggiungi i dati esclusi al test set
X_test = pd.concat([X_test, X_excluded], ignore_index=True)
y_test = pd.concat([y_test, y_excluded], ignore_index=True)

# Applica la standardizzazione ai dati
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_normal)
X_test_scaled = scaler.transform(X_test)


# Definizione della griglia di parametri
param_grid = {
    'gamma': [0.01, 0.1, 1],
    'nu': [0.01, 0.05, 0.1],  
    'kernel': ['linear', 'rbf', 'poly']
}

grid_search = GridSearchCV(
    estimator=OneClassSVM(),
    param_grid=param_grid,
    scoring=make_scorer(custom_f1),
    cv=5,
    verbose=1,
    n_jobs=-1
)

grid_search.fit(X_train_scaled, y_train_normal)

print("\nMigliori Parametri Trovati:")
print(grid_search.best_params_)

best_model = grid_search.best_estimator_

f1 = evaluate(best_model, X_test_scaled, y_test)
print(f"F1 score of the best model on test data: {f1:.2f}%")

Fitting 5 folds for each of 27 candidates, totalling 135 fits

Migliori Parametri Trovati:
{'gamma': 0.01, 'kernel': 'rbf', 'nu': 0.01}
Model Performance
Accuracy = 90.70%.
Fbeta = 62.80%.
Recall = 96.30%.
F1 = 72.22%.
Precision = 57.78%.
-----------------------------------
True Positives (TP): 26
False Positives (FP): 19
True Negatives (TN): 169
False Negatives (FN): 1
-----------------------------------
F1 score of the best model on test data: 72.22%
